# 01. 순환 신경망 (RNN & LSTM)
> 순환신경망 (RNN)에 대해 알아봅시다.

- toc: true 
- badges: true
- comments: true
- categories: [Day 7]
- permalink: /rnn_lstm
- exec: colab

### 1. RNN as Auto Regressive Model

![](https://github.com/gusdnd852/bigdata-lecture/blob/master/_notebooks/img/Day5/reg05.png?raw=true)
 
우리는 이미 회귀모델 시간에 '자기 종속성'이라는 속성에 대해 이야기 한적이 있습니다. 자기종속성이란 이전 타임스텝의 출력이 다음 타임스텝의 입력에 영향을 준다는 속성입니다. 우리가 오늘 배울 RNN이라는 신경망은 바로 이러한 자기 종속성을 다층 퍼셉트론으로 모델링한 구조입니다. <br><br>

음악, 동영상, 에세이, 시, 소스 코드, 주가 차트. 이것들의 공통점은 무엇일까요? 바로 **시간적인 순서를 갖는 시퀀스**라는 점입니다. 음악은 음계들의 시퀀스, 동영상은 이미지의 시퀀스, 에세이는 단어들의 시퀀스로 볼 수 있습니다. 시퀀스의 길이는 가변적입니다. 소설에는 한 페이지짜리 단편소설도 있고 열권짜리 장편소설도 있죠. 기존의 뉴럴 네트워크 알고리즘은 고정된 크기의 입력을 다루는 데는 탁월하지만, 가변적인 크기의 데이터를 모델링하기에는 적합하지 않습니다. <br><br>

RNN(Recurrent Neural Network, 순환신경망)은 **시퀀스 데이터를 모델링 하기 위해 등장**했습니다. RNN이 기존의 뉴럴 네트워크와 다른 점은 **‘기억’(다른 말로 hidden state)을 갖고 있다는 점**입니다. 네트워크의 기억은 지금까지의 입력 데이터를 요약한 정보라고 볼 수 있습니다. 새로운 입력이 들어올때마다 네트워크는 자신의 기억을 조금씩 수정합니다. 결국 입력을 모두 처리하고 난 후 네트워크에게 남겨진 기억은 시퀀스 전체를 요약하는 정보가 됩니다. 이는 사람이 시퀀스를 처리하는 방식과 비슷합니다. 이 글을 읽을 때도 우리는 이전까지의 단어에 대한 기억을 바탕으로 새로운 단어를 이해합니다. <br><br>

이 과정은 새로운 단어마다 계속해서 반복되기 때문에 RNN에는 Recurrent, 즉 순환적이라는 이름이 붙습니다. RNN은 이런 반복을 통해 아무리 긴 시퀀스라도 처리할 수 있는 것입니다. <br><br>

![](https://files.slack.com/files-pri/T25783BPY-F6YUKQKCP/rnn-diagram.png?pub_secret=9e9b7d3f1e)

위 다이어그램에서 빨간색 사각형은 입력, 노란색 사각형은 기억, 파란색 사각형은 출력을 나타냅니다. 첫번째 입력이 들어오면 첫번째 기억이 만들어집니다. 두번째 입력이 들어오면 기존의 기억과 새로운 입력을 참고하여 새 기억을 만듭니다. 입력의 길이만큼 이 과정을 얼마든지 반복할 수 있습니다. 각각의 기억은 그때까지의 입력을 요약해서 갖고 있는 정보입니다. RNN은 이 요약된 정보를 바탕으로 출력을 만들어 냅니다. <br><br>

### 2. What can RNNs do?

RNN의 입력과 출력은 우리가 네트워크에게 시키고 싶은 것이 무엇이냐에 따라 얼마든지 달라질 수 있습니다. 아래는 몇가지 예시입니다. <br><br>

#### 2.1. 시계열 예측(e.g. 주가 예측)
![](https://t1.daumcdn.net/cfile/tistory/99C2D5465C66628B13)

주가예측과 같은 시계열 예측 문제를 RNN으로 풀어볼 수 있습니다. 주가예측의 경우 이전 시간의 주가가 다음 시간의 주가에 영향을 주기 때문에 Auto regressive model인 RNN으로 풀어볼 수 있는 것입니다. 
<br><br>

#### 2.2. 자연어 처리(e.g. 기계 번역)

![](https://files.slack.com/files-pri/T25783BPY-F6X6W5UTA/machine-translator.png?pub_secret=c4b0c11754)

구글의 번역기와 네이버의 파파고와 같은 자연어처리 시스템들은 RNN을 응용한 모델로 만들어졌습니다. RNN 기반 모델은 기존 통계 기반 모델의 비해 우수한 성능을 낸다고 알려져 있습니다. 자연어 역시 마찬가지로 이전 단어에 의해 다음 단어가 결정되므로 RNN으로 풀 수 있는 것입니다. <br><br>

### 3. More Details in RNN

RNN은 루프가 들어있고, 과거의 데이터가 미래에 영향을 줄 수 있는 구조를 가지고 있습니다.

![](https://github.com/gusdnd852/bigdata-lecture/blob/master/_notebooks/img/Day7/00.png?raw=true)

위 다이어그램에서 하나의 뉴럴 네트워크 덩어리, A는 xt 를 입력값으로 가지고  ht를 결과값으로 내놓습니다. 이 루프는 정보가 다음 단계의 뉴럴 네트워크로 이동하게 만들어줍니다. 이러한 루프는 Recurrent Neural Network가 뭔가 불가사의하게 느끼게 만들어줍니다. 하지만, 당신이 조금만 더 생각해본다면, 이 알고리즘은 다른 뉴럴 네트워크 알고리즘과 그렇게 큰 차이가 있지 않다는 것을 알게 됩니다. Recurrent Neural Network는 하나의 네트워크가 여러개 복사된 형태를 띄고 있습니다. 각각의 네트워크는 다음 단계로 정보를 넘겨줍니다. 저 루프를 좀 풀어서 보면 아래 그림과 같은 모양을 띄게 됩니다.

![](https://github.com/gusdnd852/bigdata-lecture/blob/master/_notebooks/img/Day7/01.png?raw=true)
